# 전반적인 요약 부분을 하나의 타입으로 통일

- output의 맨 첫 번쨰 문장은 `전반적인 요약`에 해당하는 부분이다

- 하지만, output에 해당하는 문장의 유형이 6가지나 존재한다

    - `"두 화자는 이 대화에서 ~ "`
    
    - `"두 화자는 ~ "`
    - `"화자들은 ~ "`
    - `"두 사람은 ~ "`
    - `"이 대화에서는 ~ "`
    - `r"SD\d{7}(?:와|과).*SD\d{7}(?:은|는)" ~ "`
        - ex) "대화에서 화자 SD2001966와 SD2001967은 ~ "

- 이 때문에
    - 모델 입장에서는
    - 입력으로 들어온 문장을 요약하는 작업을 하기에 앞서
    - 입력으로 들어온 문장을 어떤 형태로 요약해야하는지를 판단 이후에
    - 요약을 하는 작업을 거치게 된다

- 결국,
    - 모델은 6개의 유형의 output을 적절하게 내뱉을 수 있는 능력까지 학습되어야 한다
    - 하지만, 이 6개의 유형을 구분짓는 확실한 feature간 상관관계가 보이지 않는다
    - 그렇다면, 이는 유형은 output을 생성해냄에 있어 그리 중요한 것이라 볼 수 있고
    - 가장 보편적인 형태 하나로 통일하고
    - 문제의 목적인 `전반적인 요약`을 하는 것에 집중한다면
    - 더 풀기 쉬운 문제가 될 수 있다

In [2]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
import re
from copy import deepcopy
import warnings
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)

def make_dataframe(path: str) -> pd.DataFrame:
    """
    Read a json file and return a pandas DataFrame.

    Parameters:
    path (str): Path to the json file.

    Returns:
    pd.DataFrame: DataFrame of the json file.
    """
    # Read the json file
    with open(path, 'r') as file:
        data = json.load(file)

    # Create a DataFrame
    # columns = ['id', 'conversation', 'subject_keyword', 'output']
    df = pd.DataFrame(data)
    df['conversation'] = df['input'].apply(lambda x: x['conversation'])
    df['subject_keyword'] = df['input'].apply(lambda x: x['subject_keyword'])

    # Drop the 'input' column
    df.drop('input', axis=1, inplace=True)

    # Speakers in the conversation
    df['speakers'] = df['conversation'].apply(lambda turns: list(set(turn['speaker'] for turn in turns)))

    # Reorder the columns
    df = df[['id', 'conversation', 'subject_keyword', 'speakers', 'output']]

    return df

train_df = make_dataframe('../resource/data/일상대화요약_train.json')
dev_df = make_dataframe('../resource/data/일상대화요약_dev.json')
test_df = make_dataframe('../resource/data/일상대화요약_test.json')
filtered_train_df = make_dataframe('./train.json')
filtered_dev_df = make_dataframe('./dev.json')
filtered_test_df = make_dataframe('./test.json')

In [3]:
# types와 type2 형태로 이루어진 `전반적인 요약` 부분을 모두 "두 화자는 이 대화에서"로 통일

types = ["두 화자는", "화자들은" ,"두 사람은", "이 대화에서는"] # "두 화자는 이 대화에서"
types2 = r"SD\d{7}(?:와|과).*SD\d{7}(?:은|는)"



def total_summary_generalization(df: pd.DataFrame) -> pd.DataFrame:
    """
    Generalize the `전반적인 요약` part of the DataFrame.

    Parameters:
    df (pd.DataFrame): DataFrame to generalize.

    Returns:
    pd.DataFrame: Generalized DataFrame.
    """
    # Copy the DataFrame
    df = deepcopy(df)

    # Standardize the total summary to start with '두 화자는 이 대화에서'
    def generalize_summary(summary: str) -> str:
        total_summary = summary.split('.')[0]

        if "두 화자는 이 대화에서" in total_summary:
            return summary
        elif re.search(types2, summary):
            total_summary = re.sub(r'(.*)'+types2, '두 화자는 이 대화에서', total_summary)+'.'
            return total_summary+'.'.join(summary.split('.')[1:])
        else:
            for type in types:
                if type in summary:
                    total_summary = re.sub(r'(.*)'+type, '두 화자는 이 대화에서', total_summary)+'.'
                    return total_summary+'.'.join(summary.split('.')[1:])
        
    df['output'] = df['output'].apply(generalize_summary)
    
    return df

In [7]:
total_summary_generalization(filtered_train_df)['output'].isnull().sum()

0

In [8]:
total_summary_generalization(filtered_dev_df)['output'].isnull().sum()

0

In [10]:
".".strip() =="."

True